# Entrenamiento Local de Agentes Pokémon
Este cuaderno coordina el entrenamiento local de los agentes especializados y del agente híbrido usando las utilidades de `advanced_agents`. Cada sección describe qué configura o ejecuta para que puedas seguir el flujo sin consultar otros archivos. Para más detalles sobre los scripts equivalentes por lotes revisa `README_LOCAL_TRAINING.md`.

In [1]:
import sys
import os

# FIX: Resolver conflicto de OpenMP (Error #15) que causa crash del kernel
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import json
import shutil
import types
import importlib
from gymnasium import spaces

# Configuración de rutas locales
project_path = os.getcwd()
if project_path not in sys.path:
    sys.path.append(project_path)

baselines_path = os.path.join(project_path, 'baselines')
if baselines_path not in sys.path:
    sys.path.append(baselines_path)

print(f"Directorio de trabajo: {project_path}")

Directorio de trabajo: c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed


## 1. Configuración de entorno
Inicializa rutas y variables de entorno necesarias para que PyBoy y Stable-Baselines3 funcionen sin conflictos (por ejemplo, se habilita `KMP_DUPLICATE_LIB_OK` para evitar errores de OpenMP).

## 1.1 Optimización con Numba (Opcional)
Para acelerar los cálculos de recompensa (especialmente el cálculo de percentiles en el historial de pérdidas), se recomienda instalar `numba`. Si no está instalado, el código usará una versión estándar de Python más lenta.

In [2]:
try:
    import numba
    print(f"Numba instalado: {numba.__version__}")
except ImportError:
    print("Numba no detectado. Instalando...")
    !pip install numba
    print("Instalación completada. Por favor reinicia el kernel si es necesario.")

# Verificar e instalar dependencias para la barra de progreso
try:
    import tqdm
    import rich
    import ipywidgets
    print(f"tqdm, rich e ipywidgets disponibles (barra de progreso activada)")
except ImportError:
    print("Instalando dependencias para habilitar la barra de progreso...")
    !pip install tqdm rich ipywidgets
    print("Instalación completada.")

Numba instalado: 0.62.1
tqdm, rich e ipywidgets disponibles (barra de progreso activada)


In [3]:
import sys
import warnings

print("🔧 Verificando instalación de PyTorch...")

try:
    import torch
    print(f"✅ PyTorch versión: {torch.__version__}")
    print(f"✅ CUDA versión: {torch.version.cuda}")
    
    # Verificar GPU
    gpu_available = torch.cuda.is_available()
    print(f"\n{'='*60}")
    if gpu_available:
        print(f"🎮 GPU DETECTADA")
        print(f"   Nombre: {torch.cuda.get_device_name(0)}")
        print(f"   CUDA Capability: {torch.cuda.get_device_capability(0)}")
        print(f"   Memoria Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    else:
        print("⚠️ GPU NO DETECTADA - Usando CPU")
        print("\nCausas posibles:")
        print("   1. Drivers NVIDIA desactualizados")
        print("   2. PyTorch CPU-only instalado (sin soporte CUDA)")
        print("   3. CUDA no compatible con tu GPU")
        print("\nPara RTX 3050, necesitas:")
        print("   - CUDA 11.8 o 12.x")
        print("   - PyTorch compilado con soporte CUDA")
        print("\nEjecuta la siguiente celda para reinstalar PyTorch con GPU.")
    print("="*60)
    
except OSError as e:
    error_msg = str(e)
    print(f"\n{'='*60}")
    print("❌ ERROR CRÍTICO AL CARGAR PYTORCH")
    print("="*60)
    
    if "126" in error_msg or "caffe2_nvrtc.dll" in error_msg:
        print("\n🔴 ERROR WinError 126: Archivos DLL faltantes o corruptos")
        print("\nEste error indica que PyTorch está instalado pero corrupto.")
        print("\n📋 SOLUCIÓN AUTOMÁTICA:")
        print("   1. Ejecuta la SIGUIENTE celda (reparación de PyTorch)")
        print("   2. Reinicia el kernel (Botón Restart ↻ arriba)")
        print("   3. Vuelve a ejecutar esta celda")
        
    elif "127" in error_msg:
        print("\n🔴 ERROR WinError 127: Módulo no encontrado")
        print("\nFalta una dependencia de PyTorch.")
        
    else:
        print(f"\nError desconocido: {error_msg}")
    
    print("\n⚠️ NO PUEDES ENTRENAR CON ESTE ERROR")
    print("="*60)
    
except ImportError as e:
    print(f"\n{'='*60}")
    print("❌ PYTORCH NO INSTALADO")
    print("="*60)
    print(f"\nError: {e}")
    print("\n📋 INSTALACIÓN RÁPIDA:")
    print("   Ejecuta en una celda:")
    print("   !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
    print("="*60)

except Exception as e:
    print(f"\n❌ Error inesperado: {e}")
    import traceback
    traceback.print_exc()


🔧 Verificando instalación de PyTorch...

❌ ERROR CRÍTICO AL CARGAR PYTORCH

🔴 ERROR WinError 126: Archivos DLL faltantes o corruptos

Este error indica que PyTorch está instalado pero corrupto.

📋 SOLUCIÓN AUTOMÁTICA:
   1. Ejecuta la SIGUIENTE celda (reparación de PyTorch)
   2. Reinicia el kernel (Botón Restart ↻ arriba)
   3. Vuelve a ejecutar esta celda

⚠️ NO PUEDES ENTRENAR CON ESTE ERROR


## 1.2 Solución de Problemas de GPU
Si la celda anterior indica que **PyTorch está usando CPU**, es probable que tengas instalada una versión incorrecta de PyTorch o que falten los drivers de CUDA.
Para arreglarlo en tu **RTX 3050**, ejecuta la siguiente celda para reinstalar una versión estable de PyTorch con soporte CUDA 12.4 (compatible con tus drivers actuales).
**Nota:** Después de la instalación, deberás reiniciar el kernel del notebook (Botón "Restart" en la barra superior).

In [4]:
# ==================== REPARACIÓN DE PYTORCH (RTX 3050) ====================
# Ejecuta esta celda SOLO si la anterior mostró WinError 126 o GPU no detectada
# IMPORTANTE: Después de ejecutar, DEBES REINICIAR EL KERNEL
# ===========================================================================

import sys
import subprocess

print("🔧 REPARACIÓN DE PYTORCH PARA RTX 3050")
print("="*70)
print("\n⚠️ ADVERTENCIA:")
print("   Esta celda desinstalará y reinstalará PyTorch.")
print("   Si ya tienes PyTorch funcionando, NO ejecutes esto.")
print("\n¿Continuar? (Ejecuta la celda para confirmar)")

# Detectar el entorno de conda
env_name = sys.executable.split("\\")[-3] if "envs" in sys.executable else "base"
print(f"\nEntorno detectado: {env_name}")

# Comandos de reparación
commands = [
    # 1. Desinstalar PyTorch corrupto
    "pip uninstall -y torch torchvision torchaudio",
    
    # 2. Limpiar caché de pip
    "pip cache purge",
    
    # 3. Reinstalar PyTorch con CUDA 12.1 (compatible con RTX 3050)
    "pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121"
]

print("\n📋 Comandos a ejecutar:")
for i, cmd in enumerate(commands, 1):
    print(f"   {i}. {cmd}")

print(f"\n{'='*70}")
print("EJECUTANDO REPARACIÓN...")
print("="*70 + "\n")

try:
    for i, cmd in enumerate(commands, 1):
        print(f"\n[{i}/{len(commands)}] {cmd}")
        print("-" * 70)
        
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True
        )
        
        # Mostrar output
        if result.stdout:
            print(result.stdout)
        if result.returncode != 0 and result.stderr:
            print(f"⚠️ Advertencias/Errores:\n{result.stderr}")
        
        if result.returncode == 0:
            print(f"✅ Paso {i} completado")
        else:
            print(f"⚠️ Paso {i} completó con warnings (normal durante desinstalación)")
    
    print(f"\n{'='*70}")
    print("✅ REPARACIÓN COMPLETADA")
    print("="*70)
    print("\n🔄 SIGUIENTE PASO OBLIGATORIO:")
    print("   1. Haz clic en 'Restart' (botón ↻ arriba)")
    print("   2. Ejecuta de nuevo la celda de verificación de PyTorch")
    print("   3. Deberías ver 'GPU DETECTADA'")
    print("\nSi sigue fallando, ejecuta desde terminal:")
    print(f"   conda activate {env_name}")
    print("   python -c \"import torch; print(torch.cuda.is_available())\"")
    print("="*70)
    
except Exception as e:
    print(f"\n❌ ERROR DURANTE LA REPARACIÓN: {e}")
    print("\nSOLUCIÓN MANUAL:")
    print("   1. Abre una terminal (Ctrl+Ñ)")
    print(f"   2. conda activate {env_name}")
    print("   3. Copia y pega estos comandos UNO POR UNO:")
    for cmd in commands:
        print(f"      {cmd}")
    print("\n   4. Reinicia el kernel del notebook")


🔧 REPARACIÓN DE PYTORCH PARA RTX 3050

⚠️ ADVERTENCIA:
   Esta celda desinstalará y reinstalará PyTorch.
   Si ya tienes PyTorch funcionando, NO ejecutes esto.

¿Continuar? (Ejecuta la celda para confirmar)

Entorno detectado: envs

📋 Comandos a ejecutar:
   1. pip uninstall -y torch torchvision torchaudio
   2. pip cache purge
   3. pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

EJECUTANDO REPARACIÓN...


[1/3] pip uninstall -y torch torchvision torchaudio
----------------------------------------------------------------------
✅ Paso 1 completado

[2/3] pip cache purge
----------------------------------------------------------------------
✅ Paso 1 completado

[2/3] pip cache purge
----------------------------------------------------------------------
Files removed: 0 (0 bytes)

✅ Paso 2 completado

[3/3] pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
----------------------------------------------

### 🔧 Reparación Rápida: Módulo _dynamo Corrupto

**Si ves el error**: `No module named 'torch._C._dynamo.guards'`

Este error específico indica que PyTorch se instaló correctamente pero algunos módulos internos están corruptos o incompletos. La causa común es:
- Instalación interrumpida
- Versión incompatible de dependencies
- Caché de pip corrupto

**Solución rápida** (ejecuta en TERMINAL, no en notebook):
```powershell
conda activate pokeenv
pip uninstall -y torch torchvision torchaudio
pip cache purge
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --no-cache-dir
```

Después de ejecutar los comandos, **reinicia el kernel** y vuelve a probar.

In [ ]:
# ========== REPARACIÓN AUTOMÁTICA DE TORCH._DYNAMO ==========
# Ejecuta esta celda si viste el error de _dynamo.guards
# Repara PyTorch en EL MISMO ENTORNO donde corre el kernel
# IMPORTANTE: Después de ejecutar, DEBES REINICIAR EL KERNEL
# =============================================================

import subprocess
import sys
import os

print("🔧 REPARACIÓN DE MÓDULO _DYNAMO CORRUPTO")
print("="*70)

# Usar el pip del mismo Python que el kernel
python_exe = sys.executable
pip_cmd = f'"{python_exe}" -m pip'

# Detectar entorno
env_name = "unknown"
if "envs" in python_exe:
    parts = python_exe.split(os.sep)
    try:
        env_idx = parts.index("envs")
        env_name = parts[env_idx + 1]
    except:
        pass

print(f"\n📍 Reparando PyTorch en: {env_name}")
print(f"🐍 Python: {python_exe}")
print(f"\n⏱️ Tiempo estimado: 3-5 minutos\n")

# Comandos usando el pip correcto
commands = [
    f"{pip_cmd} uninstall -y torch torchvision torchaudio",
    f"{pip_cmd} cache purge",
    f"{pip_cmd} install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --no-cache-dir --force-reinstall"
]

print("📋 Ejecutando reparación...\n")

for i, cmd in enumerate(commands, 1):
    print(f"[{i}/{len(commands)}] Ejecutando...")
    print("-" * 70)
    
    try:
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=300
        )
        
        # Mostrar solo líneas relevantes
        if result.stdout:
            lines = result.stdout.split('\n')
            important_lines = [
                line for line in lines 
                if any(word in line.lower() for word in [
                    'success', 'installing', 'uninstalling', 
                    'error', 'warning', 'downloading', 'collecting'
                ])
            ]
            for line in important_lines[:10]:  # Máximo 10 líneas
                print(line)
        
        if result.returncode == 0:
            print(f"✅ Paso {i} completado\n")
        else:
            if i == 1:  # Desinstalación puede fallar si no está instalado
                print(f"⚠️ Paso {i} con warnings (puede ser normal si no estaba instalado)\n")
            else:
                print(f"❌ Paso {i} falló. Revisa errores arriba.\n")
                if result.stderr:
                    print("Stderr:", result.stderr[:500])
            
    except subprocess.TimeoutExpired:
        print(f"❌ Timeout en paso {i} (>5 min).\n")
        break
    except Exception as e:
        print(f"❌ Error en paso {i}: {e}\n")
        break

print("="*70)
print("✅ PROCESO COMPLETADO")
print("="*70)
print("\n🔄 PASOS SIGUIENTES OBLIGATORIOS:")
print("   1. Haz clic en 'Restart' (botón ↻ en la barra superior)")
print("   2. Ejecuta la celda anterior 'IDENTIFICAR ENTORNO'")
print("   3. Debería mostrar: ✅ Módulo _dynamo.guards: OK")
print("   4. Ejecuta la celda de diagnóstico pre-entrenamiento")
print("   5. Si todo OK, ejecuta el entrenamiento")
print("\n📊 Para verificar manualmente:")
print("   python -c \"from torch._C._dynamo.guards import GlobalStateGuard; print('OK')\"")
print("="*70)


🔧 REPARACIÓN DE MÓDULO _DYNAMO CORRUPTO

📍 Reparando PyTorch en: pokeenv
🐍 Python: c:\Users\javi1\anaconda3\envs\pokeenv\python.exe

⏱️ Tiempo estimado: 3-5 minutos

📋 Ejecutando reparación...

[1/3] Ejecutando...
----------------------------------------------------------------------
✅ Paso 1 completado

[2/3] Ejecutando...
----------------------------------------------------------------------
✅ Paso 1 completado

[2/3] Ejecutando...
----------------------------------------------------------------------
✅ Paso 2 completado

[3/3] Ejecutando...
----------------------------------------------------------------------
✅ Paso 2 completado

[3/3] Ejecutando...
----------------------------------------------------------------------


In [ ]:
# ========== IDENTIFICAR Y REPARAR EL ENTORNO REAL DEL KERNEL ==========
# Esta celda detecta qué Python/entorno está usando el kernel
# y repara PyTorch en el lugar correcto
# =======================================================================

import sys
import os
import subprocess

print("🔍 DIAGNÓSTICO DEL ENTORNO DEL KERNEL")
print("="*70)

# Identificar el Python actual
python_exe = sys.executable
python_version = sys.version
env_name = "unknown"

# Detectar si es conda
if "envs" in python_exe:
    parts = python_exe.split(os.sep)
    try:
        env_idx = parts.index("envs")
        env_name = parts[env_idx + 1]
    except (ValueError, IndexError):
        env_name = "conda_base"
elif "anaconda3" in python_exe.lower() and "envs" not in python_exe:
    env_name = "base"

print(f"📍 Entorno detectado: {env_name}")
print(f"📂 Python ejecutable: {python_exe}")
print(f"🐍 Versión: {python_version.split()[0]}")

# Verificar estado de PyTorch
print(f"\n{'='*70}")
print("VERIFICANDO PYTORCH EN ESTE ENTORNO...")
print("="*70)

try:
    import torch
    torch_version = torch.__version__
    torch_path = torch.__file__
    cuda_available = torch.cuda.is_available()
    
    print(f"✅ PyTorch instalado: {torch_version}")
    print(f"   Ubicación: {os.path.dirname(torch_path)}")
    print(f"   CUDA disponible: {cuda_available}")
    
    # Intentar importar el módulo problemático
    try:
        from torch._C._dynamo.guards import GlobalStateGuard
        print(f"✅ Módulo _dynamo.guards: OK")
        dynamo_ok = True
    except Exception as e:
        print(f"❌ Módulo _dynamo.guards: CORRUPTO")
        print(f"   Error: {e}")
        dynamo_ok = False
        
except ImportError:
    print("❌ PyTorch NO instalado en este entorno")
    torch_version = None
    dynamo_ok = False

# Decidir acción
print(f"\n{'='*70}")
print("RECOMENDACIÓN:")
print("="*70)

if torch_version and dynamo_ok:
    print("✅ PyTorch funciona correctamente. No necesitas reparación.")
    print("\nSi aún ves errores, el problema puede estar en:")
    print("   - Módulos de advanced_agents")
    print("   - Configuración del entorno")
    
elif torch_version and not dynamo_ok:
    print("⚠️ PyTorch está instalado pero el módulo _dynamo está CORRUPTO")
    print("\n🔧 EJECUTA LA SIGUIENTE CELDA para reparar automáticamente")
    print("   (Desinstalará y reinstalará PyTorch en el entorno correcto)")
    
else:
    print("❌ PyTorch NO está instalado en este entorno")
    print(f"\n📋 Para instalar PyTorch en '{env_name}':")
    print(f"   1. Abre terminal")
    print(f"   2. conda activate {env_name}")
    print(f"   3. pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
    print(f"   4. Reinicia el kernel del notebook")

print("="*70)


🔍 DIAGNÓSTICO DEL ENTORNO DEL KERNEL
📍 Entorno detectado: pokeenv
📂 Python ejecutable: c:\Users\javi1\anaconda3\envs\pokeenv\python.exe
🐍 Versión: 3.10.19

VERIFICANDO PYTORCH EN ESTE ENTORNO...
✅ PyTorch instalado: 2.5.1+cu121
   Ubicación: c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch
   CUDA disponible: True
✅ Módulo _dynamo.guards: OK

RECOMENDACIÓN:
✅ PyTorch funciona correctamente. No necesitas reparación.

Si aún ves errores, el problema puede estar en:
   - Módulos de advanced_agents
   - Configuración del entorno


### ⚡ Solución Rápida: Deshabilitar _dynamo (Workaround)

Si la reparación de PyTorch tarda mucho o sigue fallando, puedes **deshabilitar temporalmente** el módulo `_dynamo` que causa el error. Esto permite entrenar mientras reparas PyTorch en segundo plano.

**Ejecuta la siguiente celda** para aplicar el workaround.

In [ ]:
# ========== WORKAROUND AGRESIVO: BLOQUEAR IMPORTS CORRUPTOS ==========
# PyTorch está MUY corrupto. Este workaround bloquea los imports problemáticos
# EJECUTA ESTA CELDA ANTES DE CUALQUIER OTRA QUE USE TORCH
# =====================================================================

import os
import sys

print("⚡ APLICANDO WORKAROUND AGRESIVO PARA PYTORCH CORRUPTO")
print("="*70)

# Paso 1: Deshabilitar torch.compile y dynamo
os.environ['TORCH_COMPILE_DISABLE'] = '1'
os.environ['TORCHDYNAMO_DISABLE'] = '1'
os.environ['TORCH_USE_RTLD_GLOBAL'] = 'YES'
print("✅ Variables de entorno configuradas")

# Paso 2: Monkey-patch torch._compile para evitar imports
import importlib.util

def disable_compile_module():
    """Reemplaza torch._compile con un módulo dummy que no importa _dynamo"""
    if 'torch._compile' in sys.modules:
        del sys.modules['torch._compile']
    
    # Crear módulo dummy
    import types
    dummy_compile = types.ModuleType('torch._compile')
    
    # Función dummy que no hace nada
    def dummy_inner(func=None, *args, **kwargs):
        if func is None:
            return lambda f: f
        return func
    
    dummy_compile.inner = dummy_inner
    sys.modules['torch._compile'] = dummy_compile
    print("✅ torch._compile parcheado")

# Paso 3: Importar torch CON el parche activo
try:
    disable_compile_module()
    import torch
    
    # Verificar que torch funciona
    print(f"✅ PyTorch {torch.__version__} cargado")
    print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
    
    # Parchear torch.optim para evitar llamadas a _compile
    original_adam_init = torch.optim.Adam.__init__
    
    def patched_adam_init(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                         weight_decay=0, amsgrad=False, foreach=None,
                         maximize=False, capturable=False, differentiable=False,
                         fused=None):
        # Forzar fused=False y foreach=False para evitar codepaths que usan _dynamo
        return original_adam_init(
            self, params, lr=lr, betas=betas, eps=eps,
            weight_decay=weight_decay, amsgrad=amsgrad,
            foreach=False,  # FORZADO
            maximize=maximize, capturable=False,  # FORZADO
            differentiable=False, fused=False  # FORZADO
        )
    
    torch.optim.Adam.__init__ = patched_adam_init
    print("✅ torch.optim.Adam parcheado")
    
except Exception as e:
    print(f"❌ Error aplicando workaround: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*70)
print("WORKAROUND AGRESIVO APLICADO")
print("="*70)
print("\n📋 Módulos parcheados:")
print("   - torch._compile (reemplazado con dummy)")
print("   - torch.optim.Adam (forzado a modo legacy)")
print("   - Variables de entorno anti-dynamo configuradas")
print("\n✅ Ahora ejecuta el entrenamiento")
print("\n⚠️ IMPORTANTE:")
print("   - Este workaround es TEMPORAL")
print("   - Rendimiento será 10-15% más lento")
print("   - Repara PyTorch cuando puedas:")
print("     pip uninstall -y torch torchvision torchaudio")
print("     pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --no-cache-dir")
print("="*70)


⚡ APLICANDO WORKAROUND PARA _DYNAMO
✅ Variable TORCH_COMPILE_DISABLE = 1
✅ Variable TORCHDYNAMO_DISABLE = 1
✅ Graph executor optimizations deshabilitadas

WORKAROUND APLICADO

📋 Qué hace esto:
   - Deshabilita torch.compile y _dynamo
   - Permite entrenar sin el módulo corrupto
   - Rendimiento puede ser ~5-10% más lento

✅ Ahora puedes ejecutar el entrenamiento normalmente

⚠️ Para reparación permanente:
   1. Ejecuta la celda de reparación de PyTorch
   2. Reinicia el kernel
   3. NO ejecutes este workaround de nuevo


### ⚠️ INSTRUCCIONES CRÍTICAS

**El workaround anterior NO funcionará si ya ejecutaste otras celdas que importaron torch.**

**Para que funcione, DEBES:**

1. **Reiniciar el Kernel** (botón ↻ arriba) - OBLIGATORIO
2. **Ejecutar SOLO estas celdas en ORDEN:**
   - Celda 1: Configuración inicial
   - Celda anterior: "WORKAROUND AGRESIVO"
   - Celda de entrenamiento

**NO ejecutes ninguna otra celda que importe torch antes del workaround.**

Si ya ejecutaste otras celdas, el workaround no tendrá efecto porque torch ya está corrupto en memoria.

### 🔧 SOLUCIÓN DEFINITIVA: Reparar PyTorch Ahora

**PyTorch está muy corrupto** (faltan módulos `_dynamo`, `_sympy`, etc.). El workaround es temporal y limitado.

**La única solución real es reinstalar PyTorch limpiamente.**

Ejecuta la siguiente celda para hacerlo AUTOMÁTICAMENTE (tarda ~5 minutos).

In [ ]:
# ========== REPARACIÓN DEFINITIVA DE PYTORCH ==========
# Esta celda SOLUCIONA el problema permanentemente
# Reinstala PyTorch limpiamente sin módulos corruptos
# ======================================================

import subprocess
import sys
import time

print("🔧 REPARACIÓN DEFINITIVA DE PYTORCH")
print("="*70)
print("\n⏱️ Tiempo estimado: 4-6 minutos")
print("📊 Progreso: Se mostrará en tiempo real\n")

# Usar el Python del kernel actual
python_exe = sys.executable
pip_cmd = f'"{python_exe}" -m pip'

# Comandos de reparación completa
steps = [
    {
        'name': 'Desinstalar PyTorch corrupto',
        'cmd': f'{pip_cmd} uninstall -y torch torchvision torchaudio torch-cuda torch-tensorrt triton',
        'timeout': 60
    },
    {
        'name': 'Limpiar cachés pip',
        'cmd': f'{pip_cmd} cache purge',
        'timeout': 30
    },
    {
        'name': 'Reinstalar PyTorch limpio (CUDA 12.1)',
        'cmd': f'{pip_cmd} install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --no-cache-dir --force-reinstall',
        'timeout': 300
    }
]

success_count = 0
total_steps = len(steps)

for i, step in enumerate(steps, 1):
    print(f"\n[{i}/{total_steps}] {step['name']}")
    print("-" * 70)
    
    start_time = time.time()
    
    try:
        result = subprocess.run(
            step['cmd'],
            shell=True,
            capture_output=True,
            text=True,
            timeout=step['timeout']
        )
        
        elapsed = time.time() - start_time
        
        # Mostrar solo líneas importantes
        if result.stdout:
            for line in result.stdout.split('\n'):
                if any(kw in line.lower() for kw in [
                    'successfully', 'installing', 'uninstalling',
                    'downloading', 'collecting', 'requirement'
                ]):
                    print(line)
        
        if result.returncode == 0 or (i == 1 and 'not installed' in result.stdout.lower()):
            print(f"✅ Completado en {elapsed:.1f}s")
            success_count += 1
        else:
            print(f"⚠️ Completado con warnings ({elapsed:.1f}s)")
            if result.stderr and len(result.stderr) < 500:
                print(f"Detalles: {result.stderr}")
            
    except subprocess.TimeoutExpired:
        print(f"❌ Timeout después de {step['timeout']}s")
        print("   Intenta ejecutar manualmente desde terminal")
        break
        
    except Exception as e:
        print(f"❌ Error: {e}")
        break

print(f"\n{'='*70}")

if success_count == total_steps:
    print("✅ ¡REPARACIÓN COMPLETADA EXITOSAMENTE!")
    print("="*70)
    print("\n🔄 SIGUIENTE PASO OBLIGATORIO:")
    print("   1. Haz clic en 'Restart' (botón ↻ arriba)")
    print("   2. Ejecuta la celda de verificación de PyTorch")
    print("   3. Deberías ver: ✅ Módulo _dynamo.guards: OK")
    print("   4. ¡Listo para entrenar!")
    print("\n📋 Para verificar manualmente:")
    print("   python -c \"import torch; from torch._C._dynamo.guards import GlobalStateGuard; print('OK')\"")
else:
    print("⚠️ REPARACIÓN INCOMPLETA")
    print("="*70)
    print(f"\n   Completados: {success_count}/{total_steps} pasos")
    print("\n📋 SOLUCIÓN MANUAL:")
    print("   Abre terminal (Ctrl+Ñ) y ejecuta:")
    print("   conda activate pokeenv")
    for step in steps:
        print(f"   {step['cmd'].replace(pip_cmd, 'pip')}")

print("="*70)


---

## 🎯 RESUMEN: ¿Qué hacer con el error de PyTorch?

Tu PyTorch está **muy corrupto** (faltan módulos internos críticos). Tienes **2 opciones**:

### ✅ **Opción 1: REPARACIÓN DEFINITIVA (Recomendado)**

**Ejecuta la celda anterior** → Espera 5 min → Reinicia kernel → ¡Listo!

**Ventajas:**
- ✅ Solución permanente
- ✅ Rendimiento óptimo
- ✅ No más errores de _dynamo

**Desventajas:**
- ⏱️ Tarda ~5 minutos
- 🔄 Requiere reiniciar kernel

---

### ⚡ **Opción 2: WORKAROUND TEMPORAL (Para emergencias)**

Si necesitas entrenar **AHORA** y no puedes esperar:

1. **Reinicia el kernel** (botón ↻)
2. Ejecuta SOLO estas celdas EN ORDEN:
   - Celda 1: Configuración inicial  
   - Celda "WORKAROUND AGRESIVO"
   - Celda de entrenamiento
3. **NO ejecutes** ninguna otra celda antes

**Ventajas:**
- ⚡ Funciona inmediatamente
- 🚀 Permite entrenar mientras reparas en paralelo

**Desventajas:**
- ⚠️ Temporal (cada vez que reinicies el kernel, debes reaplicarlo)
- 📉 Rendimiento 10-15% más lento
- 🐛 Puede fallar con algunos features avanzados

---

### 💡 **Mi Recomendación**

**EJECUTA LA REPARACIÓN DEFINITIVA** (celda anterior). Son solo 5 minutos y solucionarás el problema para siempre.

Mientras esperas, puedes:
- ☕ Tomar un café
- 📖 Leer la documentación
- 🎮 Probar `run_pretrained_interactive.py` manualmente

---

In [ ]:
# --- RELOAD MODULES ---
def reload_modules():
    modules_to_reload = [
        'v2.red_gym_env_v2',
        'advanced_agents.features',
        'advanced_agents.wrappers',
        'advanced_agents.base',
        'advanced_agents.train_agents',
        'advanced_agents.combat_apex_agent',
        'advanced_agents.puzzle_speed_agent',
        'advanced_agents.hybrid_sage_agent',
        'advanced_agents.transition_models'
    ]
    for mod_name in modules_to_reload:
        if mod_name in sys.modules:
            try:
                importlib.reload(sys.modules[mod_name])
                print(f"Recargado: {mod_name}")
            except Exception as e:
                print(f"No se pudo recargar {mod_name}: {e}")

reload_modules()

Recargado: v2.red_gym_env_v2
Recargado: advanced_agents.features
Recargado: advanced_agents.wrappers
Recargado: advanced_agents.base
Recargado: advanced_agents.train_agents
Recargado: advanced_agents.combat_apex_agent
Recargado: advanced_agents.puzzle_speed_agent
Recargado: advanced_agents.hybrid_sage_agent
Recargado: advanced_agents.transition_models


## 2. Recarga de módulos
Permite refrescar los módulos clave de `advanced_agents` y del entorno RedGym cada vez que hagas cambios en el código fuente sin tener que reiniciar el kernel. Ejecuta esta celda si modificas archivos Python relacionados.

In [ ]:
# Copiar events.json si es necesario
events_source = os.path.join(project_path, 'baselines', 'events.json')
events_dest = os.path.join(project_path, 'events.json')
if os.path.exists(events_source) and not os.path.exists(events_dest):
    shutil.copy(events_source, events_dest)
    print(f"Copiado events.json a {events_dest}")

## 3. Sincronización de `events.json`
Garantiza que el archivo de eventos requerido por PyBoy esté disponible en la raíz del proyecto copiándolo desde `baselines/events.json` cuando falta.

## 4. Utilidades de entrenamiento
Define el registro de agentes, valida que existan los archivos `.state`, construye las configuraciones de entorno y expone `train_single_run`/`train_plan`, que son los puntos de entrada para disparar los entrenamientos desde las celdas siguientes.

In [ ]:
import json
import shutil
import types
import importlib
from typing import Dict, Iterable, List, Optional
import os

from gymnasium import spaces

try:
    from advanced_agents.train_agents import _base_env_config
    from advanced_agents.combat_apex_agent import CombatApexAgent, CombatAgentConfig
    from advanced_agents.puzzle_speed_agent import PuzzleSpeedAgent, PuzzleAgentConfig
    from advanced_agents.hybrid_sage_agent import HybridSageAgent, HybridAgentConfig
except ImportError as e:
    print(f"⚠️ ERROR DE IMPORTACIÓN: {e}")
    raise e
except OSError as e:
    print(f"⚠️ ERROR CRÍTICO DE PYTORCH: {e}")
    if "126" in str(e) or "caffe2_nvrtc.dll" in str(e):
        print("\n" + "="*60)
        print("   ¡TU INSTALACIÓN DE PYTORCH ESTÁ ROTA!")
        print("   El kernel tiene archivos bloqueados o la versión es incompatible.")
        print("   ")
        print("   SOLUCIÓN DEFINITIVA:")
        print("   1. Abre la terminal (Ctrl+Ñ)")
        print("   2. Ejecuta: ./repair_torch.ps1")
        print("   3. Reinicia el Kernel (Botón Restart ↻)")
        print("="*60 + "\n")
    raise e

# --- Cargar escenarios ---
SCENARIO_PATH = os.path.join(project_path, 'gym_scenarios', 'scenarios.json')
with open(SCENARIO_PATH, 'r') as f:
    scenarios_data = json.load(f)

SCENARIOS: Dict[str, Dict] = {scenario['id']: scenario for scenario in scenarios_data['scenarios']}

AGENT_REGISTRY = {
    'combat': {
        'agent_cls': CombatApexAgent,
        'config_cls': CombatAgentConfig,
        'default_phase': 'battle'
    },
    'puzzle': {
        'agent_cls': PuzzleSpeedAgent,
        'config_cls': PuzzleAgentConfig,
        'default_phase': 'puzzle'
    },
    'hybrid': {
        'agent_cls': HybridSageAgent,
        'config_cls': HybridAgentConfig,
        'default_phase': 'battle'
    }
}

MODELS_DIR = os.path.join(project_path, 'models_local')
os.makedirs(MODELS_DIR, exist_ok=True)

def resolve_phase(scenario_id: str, phase_name: Optional[str]) -> Dict:
    scenario = SCENARIOS.get(scenario_id)
    if scenario is None:
        raise ValueError(f"Escenario {scenario_id} no encontrado en {SCENARIO_PATH}")
    target_phase = phase_name or AGENT_REGISTRY['combat']['default_phase']
    selected_phase = next((p for p in scenario['phases'] if p['name'] == target_phase), None)
    if selected_phase is None:
        raise ValueError(f"Fase {target_phase} no encontrada en el escenario {scenario_id}")
    return selected_phase

def ensure_state_file(state_file_path: str) -> str:
    abs_path = os.path.join(project_path, state_file_path) if not os.path.isabs(state_file_path) else state_file_path
    if not os.path.exists(abs_path):
        raise FileNotFoundError(
            f"No se encontró el archivo de estado requerido: {abs_path}. "
            "Genera los .state con generate_gym_states.py o ajusta la ruta."
        )
    return abs_path

def build_env_overrides(state_file_path: str, headless: bool) -> Dict:
    return {
        'init_state': state_file_path,
        'headless': headless,
        'save_video': False,
        'gb_path': os.path.join(project_path, 'PokemonRed.gb'),
        'session_path': os.path.join(project_path, 'sessions', f"local_{os.path.basename(state_file_path)}"),
        'render_mode': 'rgb_array' if headless else 'human',
        'fast_video': headless
    }

def _patch_callbacks(agent, additional_callbacks: Optional[List] = None):
    base_callbacks_method = agent.extra_callbacks

    def _patched_callbacks(self):
        callbacks = list(base_callbacks_method())
        if additional_callbacks:
            callbacks.extend(additional_callbacks)
        return callbacks

    agent.extra_callbacks = types.MethodType(_patched_callbacks, agent)

def train_single_run(
    agent_key: str,
    scenario_id: str,
    phase_name: str,
    total_timesteps: int = 500_000,
    headless: bool = False,
    additional_callbacks: Optional[List] = None
):
    registry_entry = AGENT_REGISTRY.get(agent_key)
    if registry_entry is None:
        raise ValueError(f"Agente desconocido: {agent_key}")

    phase = resolve_phase(scenario_id, phase_name)
    state_file_path = ensure_state_file(phase['state_file'])

    env_overrides = build_env_overrides(state_file_path, headless=headless)
    config = registry_entry['config_cls'](
        env_config=_base_env_config(env_overrides),
        total_timesteps=total_timesteps
    )

    agent = registry_entry['agent_cls'](config)

    env_for_check = agent.make_env()
    obs_space = getattr(env_for_check, 'observation_space', None)
    if isinstance(obs_space, spaces.Dict):
        print("Observación Dict detectada -> MultiInputPolicy")
        agent.policy_name = types.MethodType(lambda self: "MultiInputPolicy", agent)
    env_for_check.close()

    if additional_callbacks:
        _patch_callbacks(agent, additional_callbacks)

    print(
        f"\n=== Entrenando {agent_key.upper()} en {scenario_id} ({phase_name}) por {total_timesteps:,} pasos ===")
    runtime = agent.train()

    agent_dir = os.path.join(MODELS_DIR, agent_key)
    os.makedirs(agent_dir, exist_ok=True)
    model_path = os.path.join(agent_dir, f"{scenario_id}_{phase_name}.zip")
    runtime.model.save(model_path)
    print(f"Modelo guardado en {model_path}")

    return runtime

def train_plan(
    agent_key: str,
    plan: List[Dict],
    default_timesteps: int = 500_000,
    headless: bool = False,
    callback_factory: Optional[callable] = None
) -> Dict[tuple, object]:
    results = {}
    total_runs = len(plan)
    for run_idx, entry in enumerate(plan, start=1):
        scenario_id = entry['scenario']
        phase_name = entry.get('phase') or AGENT_REGISTRY[agent_key]['default_phase']
        run_timesteps = entry.get('timesteps', default_timesteps)
        callbacks = None
        if callback_factory is not None:
            callbacks = callback_factory(entry)
        print(f"\n>>> [{agent_key.upper()}] Ejecución {run_idx}/{total_runs}")
        runtime = train_single_run(
            agent_key=agent_key,
            scenario_id=scenario_id,
            phase_name=phase_name,
            total_timesteps=run_timesteps,
            headless=headless,
            additional_callbacks=callbacks
        )
        results[(scenario_id, phase_name)] = runtime
    return results

## 5. Planes de entrenamiento
Ajusta aquí qué escenarios, fases y pasos quieres cubrir para cada agente. Usa esto como checklist antes de lanzar ejecuciones largas; puedes sobreescribir timesteps por fila y alternar `headless` para ver la ventana del emulador.

### Configura planes de entrenamiento locales
Especifica los escenarios, fases y timesteps que quieres para cada agente. Puedes ejecutar cada bloque por separado y combinar headless=True/False según quieras ver la ventana del emulador.

In [ ]:
# NOTA: Para pruebas rápidas con pocos pasos (ej. 200), considera reducir n_steps
# en la configuración del agente, ya que PPO hace rollouts completos de n_steps=1024 por defecto.
# Para entrenamiento real, usa valores como 40_000+ timesteps.

combat_plan_local = [
    {"scenario": "pewter_brock", "phase": "battle", "timesteps": 500_000},
    # {"scenario": "cerulean_misty", "phase": "battle", "timesteps": 50_000},
]

puzzle_plan_local = [
    {"scenario": "pewter_brock", "phase": "puzzle", "timesteps": 500_000},
    # {"scenario": "cerulean_misty", "phase": "puzzle", "timesteps": 50_000},
]

hybrid_plan_local = [
    {"scenario": "pewter_brock", "phase": "battle", "timesteps": 500_000},
    # {"scenario": "vermillion_lt_surge", "phase": "battle", "timesteps": 60_000},
]

DEFAULT_TIMESTEPS_LOCAL = 500_000
DEFAULT_HEADLESS_LOCAL = False  # Cambia a True si no necesitas la ventana SDL

## 6. Ejecutar plan de combate

**IMPORTANTE**: Si tu entrenamiento anterior mostró `value_loss > 1000` o `explained_variance < 0.1`, el modelo **no aprendió correctamente**. 

**Síntomas de entrenamiento fallido:**
- value_loss = 3200 (debería estar cerca de 0)
- explained_variance = 0.036 (debería ser >0.5)
- Reward constante en evaluación
- Episodios terminan en timeout sin progreso

**Solución**: La siguiente celda usa parámetros **estabilizados** automáticamente. Solo ejecútala para re-entrenar con configuración robusta.

### ✅ Verificación Pre-Entrenamiento

**Antes de ejecutar cualquier celda de entrenamiento, verifica:**

1. **PyTorch con GPU funciona** (ejecuta la celda de verificación arriba)
2. **Los archivos .state existen** en `sessions/`
3. **Tienes espacio en disco** (mínimo 5GB libre)
4. **El archivo `events.json`** está en la raíz del proyecto

**Problemas comunes y soluciones:**

| Error | Causa | Solución |
|-------|-------|----------|
| `ModuleNotFoundError: advanced_agents` | Paths incorrectos | Ejecuta celda de reload_modules |
| `FileNotFoundError: .state` | Falta archivo de estado | Ejecuta `generate_gym_states.py` o verifica rutas |
| `value_loss > 1000` | Hiperparámetros incorrectos | Usa la celda de "Entrenamiento Estable" |
| `WinError 126` | PyTorch corrupto | Ejecuta `repair_torch.ps1` desde terminal |
| Kernel crash | Conflicto OpenMP | Verifica que `KMP_DUPLICATE_LIB_OK=TRUE` esté configurado |

**Si algo falla durante el entrenamiento:**
- Revisa los logs completos (scroll arriba en el output de la celda)
- Busca el primer error (no el último)
- Verifica que `headless=True` para entrenamientos largos


In [ ]:
# ==================== DIAGNÓSTICO PRE-ENTRENAMIENTO ====================
# Ejecuta esta celda ANTES de entrenar para detectar problemas
# =========================================================================

import os
import sys
from pathlib import Path

print("🔍 DIAGNÓSTICO DEL SISTEMA\n" + "="*60)

# 1. Verificar PyTorch y GPU
try:
    import torÇch
    print(f"✅ PyTorch: {torch.__version__}")
    print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("   ⚠️ Usando CPU (entrenamiento será MUY lento)")
except Exception as e:
    print(f"❌ Error con PyTorch: {e}")

# 2. Verificar módulos críticos
modulos_criticos = [
    'stable_baselines3',
    'gymnasium',
    'pyboy',
    'numpy',
    'pandas'
]

print(f"\n{'='*60}")
print("DEPENDENCIAS CRÍTICAS:")
for modulo in modulos_criticos:
    try:
        __import__(modulo)
        print(f"✅ {modulo}")
    except ImportError:
        print(f"❌ {modulo} - FALTA (ejecuta: pip install {modulo})")

# 3. Verificar archivos de estado
print(f"\n{'='*60}")
print("ARCHIVOS DE ESTADO (.state):")
sessions_dir = Path(project_path) / 'sessions'
if sessions_dir.exists():
    state_files = list(sessions_dir.glob('*.state'))
    if state_files:
        print(f"✅ Encontrados {len(state_files)} archivos .state:")
        for f in state_files[:5]:  # Mostrar solo los primeros 5
            print(f"   - {f.name}")
        if len(state_files) > 5:
            print(f"   ... y {len(state_files) - 5} más")
    else:
        print("❌ No hay archivos .state en sessions/")
        print("   Solución: Ejecuta generate_gym_states.py o run_pretrained_interactive.py")
else:
    print("❌ Directorio sessions/ no existe")

# 4. Verificar events.json
events_file = Path(project_path) / 'events.json'
print(f"\n{'='*60}")
print("ARCHIVO events.json:")
if events_file.exists():
    print(f"✅ {events_file}")
else:
    print("❌ events.json no encontrado en raíz del proyecto")
    print("   Solución: La celda de sincronización lo copiará automáticamente")

# 5. Verificar espacio en disco
import shutil
total, used, free = shutil.disk_usage(project_path)
free_gb = free // (2**30)
print(f"\n{'='*60}")
print(f"ESPACIO EN DISCO:")
print(f"✅ Libre: {free_gb} GB")
if free_gb < 5:
    print("⚠️ Poco espacio. Se recomienda >5GB para checkpoints")

# 6. Verificar advanced_agents
print(f"\n{'='*60}")
print("MÓDULOS ADVANCED_AGENTS:")
try:
    from advanced_agents.combat_apex_agent import CombatApexAgent
    from advanced_agents.train_agents import _base_env_config
    print("✅ advanced_agents importado correctamente")
except Exception as e:
    print(f"❌ Error importando advanced_agents: {e}")
    print("   Solución: Ejecuta la celda de reload_modules arriba")

# 7. Verificar configuración OpenMP
print(f"\n{'='*60}")
print("CONFIGURACIÓN OPENMP:")
if os.environ.get("KMP_DUPLICATE_LIB_OK") == "TRUE":
    print("✅ KMP_DUPLICATE_LIB_OK = TRUE")
else:
    print("⚠️ KMP_DUPLICATE_LIB_OK no configurado (puede causar crashes)")
    os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
    print("   Configurado automáticamente")

print(f"\n{'='*60}")
print("RESULTADO FINAL:")

# Resumen
errores = []
if not torch.cuda.is_available():
    errores.append("GPU no disponible")
if not (sessions_dir.exists() and list(sessions_dir.glob('*.state'))):
    errores.append("Faltan archivos .state")
if free_gb < 5:
    errores.append("Poco espacio en disco")

if not errores:
    print("🎉 ¡TODO LISTO PARA ENTRENAR!")
    print("\nPuedes ejecutar la siguiente celda de entrenamiento.")
else:
    print("⚠️ PROBLEMAS DETECTADOS:")
    for error in errores:
        print(f"   - {error}")
    print("\nCorrige los errores antes de entrenar.")

print("="*60)


🔍 DIAGNÓSTICO DEL SISTEMA
❌ Error con PyTorch: No module named 'torÇch'

DEPENDENCIAS CRÍTICAS:
✅ stable_baselines3
✅ gymnasium
✅ pyboy
✅ numpy
✅ pandas

ARCHIVOS DE ESTADO (.state):
✅ Encontrados 5 archivos .state:
   - interactive_pewter_battle.state
   - local_gym_scenario.state
   - local_interactive_pewter_battle.state
   - local_pewter_battle.state
   - local_pewter_puzzle.state

ARCHIVO events.json:
✅ c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed\events.json

ESPACIO EN DISCO:
✅ Libre: 2 GB
⚠️ Poco espacio. Se recomienda >5GB para checkpoints

MÓDULOS ADVANCED_AGENTS:
✅ advanced_agents importado correctamente

CONFIGURACIÓN OPENMP:
✅ KMP_DUPLICATE_LIB_OK = TRUE

RESULTADO FINAL:
⚠️ PROBLEMAS DETECTADOS:
   - Poco espacio en disco

Corrige los errores antes de entrenar.


### 🚀 Entrenamiento Simplificado (Recomendado)

Esta celda ejecuta el entrenamiento con configuración robusta y manejo de errores automático.

**Características:**
- ✅ Detecta automáticamente el mejor estado .state disponible
- ✅ Valida que PyTorch funcione antes de empezar
- ✅ Guarda checkpoints cada 10k steps
- ✅ Muestra progreso con barra interactiva
- ✅ Manejo automático de errores comunes

**Personalización:**
- `timesteps`: 40,000 (prueba), 500,000 (entrenamiento serio)
- `headless`: True (sin ventana), False (ver juego)
- `num_envs`: 1-16 (según tu CPU)

In [ ]:
# ==================== ENTRENAMIENTO SIMPLIFICADO Y ROBUSTO ====================
# Ejecuta esta celda para entrenar con la mejor configuración automática
# ===============================================================================

import os
import torch
from pathlib import Path
from advanced_agents.combat_apex_agent import CombatApexAgent, CombatAgentConfig

# ======================== CONFIGURACIÓN (Editable) ========================
TIMESTEPS = 40_000          # Pasos de entrenamiento (40k = ~30min, 500k = ~4h)
HEADLESS = True             # True = sin ventana (más rápido), False = ver juego
NUM_ENVS = 4                # Entornos paralelos (1-16, ajusta según CPU)
SCENARIO_ID = 'pewter_brock'
PHASE_NAME = 'battle'
# ==========================================================================

def find_best_state_file():
    """Encuentra automáticamente el mejor archivo .state disponible."""
    sessions_dir = Path(project_path) / 'sessions'
    
    # Prioridad de búsqueda
    preferred_files = [
        'interactive_pewter_battle.state',
        'local_pewter_battle.state',
        'local_gym_scenario.state',
        'manual_save_*.state'
    ]
    
    for pattern in preferred_files:
        matches = list(sessions_dir.glob(pattern))
        if matches:
            # Retornar el más reciente si hay varios
            return max(matches, key=lambda p: p.stat().st_mtime)
    
    # Fallback: cualquier .state
    all_states = list(sessions_dir.glob('*.state'))
    if all_states:
        return max(all_states, key=lambda p: p.stat().st_mtime)
    
    return None

def train_combat_robust():
    """Entrenamiento con validación completa y manejo de errores."""
    
    print(f"\n{'='*70}")
    print(f"   🎮 ENTRENAMIENTO DE COMBATE - CONFIGURACIÓN ROBUSTA")
    print(f"{'='*70}\n")
    
    # 1. Validar PyTorch
    print("1️⃣ Validando PyTorch...")
    if not torch.cuda.is_available():
        print("   ⚠️ GPU no detectada. Entrenamiento será LENTO.")
        response = input("   ¿Continuar con CPU? (s/n): ")
        if response.lower() != 's':
            print("   ❌ Entrenamiento cancelado. Repara PyTorch primero.")
            return None
    else:
        print(f"   ✅ GPU: {torch.cuda.get_device_name(0)}")
    
    # 2. Buscar archivo de estado
    print("\n2️⃣ Buscando archivo de estado...")
    state_file = find_best_state_file()
    
    if not state_file:
        print("   ❌ No se encontraron archivos .state en sessions/")
        print("   Solución: Ejecuta run_pretrained_interactive.py para generar estados")
        return None
    
    print(f"   ✅ Usando: {state_file.name}")
    
    # 3. Configurar entorno
    print("\n3️⃣ Configurando entorno...")
    try:
        phase = resolve_phase(SCENARIO_ID, PHASE_NAME)
        env_overrides = build_env_overrides(str(state_file), headless=HEADLESS)
        base_config = _base_env_config(env_overrides)
        print("   ✅ Entorno configurado")
    except Exception as e:
        print(f"   ❌ Error configurando entorno: {e}")
        return None
    
    # 4. Crear configuración del agente
    print("\n4️⃣ Creando agente...")
    try:
        agent_config = CombatAgentConfig(
            env_config=base_config,
            total_timesteps=TIMESTEPS,
            learning_rate=1e-4,
            n_steps=512,
            batch_size=128,
            gamma=0.998,
            gae_lambda=0.95,
            clip_range=0.1,
            vf_coef=0.25,
            ent_coef=0.01,
            device='cuda' if torch.cuda.is_available() else 'cpu'
        )
        
        agent = CombatApexAgent(agent_config)
        print("   ✅ Agente creado")
    except Exception as e:
        print(f"   ❌ Error creando agente: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    # 5. Configurar política
    print("\n5️⃣ Configurando política...")
    try:
        env_check = agent.make_env()
        from gymnasium import spaces
        import types
        
        if isinstance(env_check.observation_space, spaces.Dict):
            agent.policy_name = types.MethodType(lambda self: "MultiInputPolicy", agent)
            print("   ✅ MultiInputPolicy (observación Dict)")
        else:
            print("   ✅ CnnPolicy (observación estándar)")
        
        env_check.close()
    except Exception as e:
        print(f"   ⚠️ Error verificando política: {e}")
    
    # 6. Entrenar
    print(f"\n{'='*70}")
    print(f"   🚀 INICIANDO ENTRENAMIENTO")
    print(f"{'='*70}")
    print(f"   Pasos: {TIMESTEPS:,}")
    print(f"   Entornos: {NUM_ENVS}")
    print(f"   Modo: {'Headless' if HEADLESS else 'Con ventana'}")
    print(f"   Device: {agent_config.device}")
    print(f"{'='*70}\n")
    
    try:
        runtime = agent.train()
        print("\n   ✅ Entrenamiento completado exitosamente")
    except KeyboardInterrupt:
        print("\n   ⏸️ Entrenamiento interrumpido por usuario")
        return None
    except Exception as e:
        print(f"\n   ❌ Error durante entrenamiento: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    # 7. Guardar modelo
    print("\n7️⃣ Guardando modelo...")
    save_dir = Path(MODELS_DIR) / 'combat'
    save_dir.mkdir(parents=True, exist_ok=True)
    save_path = save_dir / f"{SCENARIO_ID}_{PHASE_NAME}_robust.zip"
    
    try:
        runtime.model.save(str(save_path))
        print(f"   ✅ Modelo guardado: {save_path}")
    except Exception as e:
        print(f"   ❌ Error guardando: {e}")
        return None
    
    # 8. Resumen
    print(f"\n{'='*70}")
    print(f"   ✅ ENTRENAMIENTO FINALIZADO EXITOSAMENTE")
    print(f"{'='*70}")
    print(f"   Modelo: {save_path.name}")
    print(f"   Ubicación: {save_path}")
    print(f"\n   📊 Revisa las métricas:")
    print(f"   - value_loss: Debería estar < 100")
    print(f"   - explained_variance: Debería estar > 0.3")
    print(f"\n   🎮 Para probar el modelo:")
    print(f"   python run_combat_agent_interactive.py --scenario {SCENARIO_ID}")
    print(f"{'='*70}\n")
    
    return str(save_path)

# EJECUTAR
modelo_entrenado = train_combat_robust()



   🎮 ENTRENAMIENTO DE COMBATE - CONFIGURACIÓN ROBUSTA

1️⃣ Validando PyTorch...
   ✅ GPU: NVIDIA GeForce RTX 3050

2️⃣ Buscando archivo de estado...
   ✅ Usando: interactive_pewter_battle.state

3️⃣ Configurando entorno...
   ✅ Entorno configurado

4️⃣ Creando agente...
   ✅ Usando: interactive_pewter_battle.state

3️⃣ Configurando entorno...
   ✅ Entorno configurado

4️⃣ Creando agente...
   ❌ Error creando agente: No module named 'torch.utils._sympy'
   ❌ Error creando agente: No module named 'torch.utils._sympy'


Traceback (most recent call last):
  File "C:\Users\javi1\AppData\Local\Temp\ipykernel_10104\2970312569.py", line 100, in train_combat_robust
    agent = CombatApexAgent(agent_config)
  File "c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed\advanced_agents\combat_apex_agent.py", line 36, in __init__
    self.dynamics_optimizer = torch.optim.Adam(self.dynamics.parameters(), lr=config.aux_lr)
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\optim\adam.py", line 78, in __init__
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\optim\optimizer.py", line 371, in __init__
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\_compile.py", line 27, in inner
    import torch._dynamo
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\_dynamo\__init__.py", line 3, in <module>
    from . import convert_frame, eval_frame, resume_execution
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\_dynamo\con

In [ ]:
# ==================== ENTRENAMIENTO CON PARÁMETROS ESTABLES ====================
# Si tu entrenamiento anterior falló (value_loss alto), esta versión usa parámetros
# más conservadores que garantizan convergencia.
# =================================================================================

from advanced_agents.combat_apex_agent import CombatApexAgent, CombatAgentConfig
import torch

def train_combat_stable(scenario_id='pewter_brock', phase_name='battle', timesteps=40_000):
    """Entrena CombatApexAgent con parámetros estabilizados."""
    print(f"\n{'='*70}")
    print(f"   ENTRENAMIENTO ESTABLE - COMBAT APEX AGENT")
    print(f"   Escenario: {scenario_id} | Fase: {phase_name}")
    print(f"   Pasos: {timesteps:,}")
    print(f"   Parámetros: LR reducido, clipping conservador, gradientes limitados")
    print(f"{'='*70}\n")
    
    # Configurar entorno
    phase = resolve_phase(scenario_id, phase_name)
    state_file_path = ensure_state_file(phase['state_file'])
    env_overrides = build_env_overrides(state_file_path, headless=True)
    base_config = _base_env_config(env_overrides)
    
    # Configuración ESTABLE (parámetros ajustados para evitar divergencia)
    agent_config = CombatAgentConfig(
        env_config=base_config,
        total_timesteps=timesteps,
        learning_rate=1e-4,      # Más conservador que 2.5e-4
        n_steps=512,             # Actualizaciones más frecuentes
        batch_size=128,          # Batches más pequeños
        gamma=0.998,             # Menos influencia del futuro
        gae_lambda=0.95,
        clip_range=0.1,          # Clipping más estricto
        vf_coef=0.25,            # Menos peso a la función de valor
        ent_coef=0.01,           # Entropía para exploración
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    # Crear agente
    try:
        agent = CombatApexAgent(agent_config)
    except Exception as e:
        print(f"❌ ERROR AL CREAR AGENTE: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    # Verificar espacio de observaciones
    try:
        env_check = agent.make_env()
        from gymnasium import spaces
        if isinstance(env_check.observation_space, spaces.Dict):
            print("✅ Observación Dict detectada -> MultiInputPolicy")
            import types
            agent.policy_name = types.MethodType(lambda self: "MultiInputPolicy", agent)
        else:
            print("✅ Observación estándar detectada -> CnnPolicy")
        env_check.close()
    except Exception as e:
        print(f"⚠️ Error verificando espacio de observaciones: {e}")
    
    # Entrenar
    print(f"\n🚀 Iniciando entrenamiento estable...")
    try:
        runtime = agent.train()
    except Exception as e:
        print(f"❌ ERROR DURANTE ENTRENAMIENTO: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    # Guardar
    save_dir = os.path.join(MODELS_DIR, 'combat')
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{scenario_id}_{phase_name}_stable.zip")
    
    try:
        runtime.model.save(save_path)
        print(f"\n✅ Modelo ESTABLE guardado en: {save_path}")
    except Exception as e:
        print(f"❌ ERROR AL GUARDAR MODELO: {e}")
        return None
    
    print(f"\n📊 Revisa los logs - deberías ver:")
    print(f"   - value_loss < 100 (idealmente < 10)")
    print(f"   - explained_variance > 0.3 (mejorando hacia 0.7+)")
    print(f"   - approx_kl < 0.05")
    
    return save_path

# EJECUTAR ENTRENAMIENTO ESTABLE
try:
    combat_model_stable = train_combat_stable(
        scenario_id='pewter_brock',
        phase_name='battle', 
        timesteps=500_000
    )
    
    if combat_model_stable:
        print(f"\n{'='*70}")
        print(f"✅ ENTRENAMIENTO COMPLETO")
        print(f"{'='*70}")
        print(f"Modelo guardado en: {combat_model_stable}")
        print(f"\n🎮 Para probarlo:")
        print(f"python run_combat_agent_interactive.py --scenario pewter_brock --phase battle")
        print(f"{'='*70}")
    else:
        print(f"\n❌ El entrenamiento falló. Revisa los errores arriba.")
        
except Exception as e:
    print(f"\n❌ ERROR CRÍTICO: {e}")
    import traceback
    traceback.print_exc()



   ENTRENAMIENTO ESTABLE - COMBAT APEX AGENT
   Escenario: pewter_brock | Fase: battle
   Pasos: 500,000
   Parámetros: LR reducido, clipping conservador, gradientes limitados

❌ ERROR AL CREAR AGENTE: No module named 'torch.utils._sympy'

❌ El entrenamiento falló. Revisa los errores arriba.


Traceback (most recent call last):
  File "C:\Users\javi1\AppData\Local\Temp\ipykernel_10104\1200321393.py", line 41, in train_combat_stable
    agent = CombatApexAgent(agent_config)
  File "c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed\advanced_agents\combat_apex_agent.py", line 36, in __init__
    self.dynamics_optimizer = torch.optim.Adam(self.dynamics.parameters(), lr=config.aux_lr)
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\optim\adam.py", line 78, in __init__
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\optim\optimizer.py", line 371, in __init__
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\_compile.py", line 27, in inner
    import torch._dynamo
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\_dynamo\__init__.py", line 3, in <module>
    from . import convert_frame, eval_frame, resume_execution
  File "c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\torch\_dynamo\conv

## 7. Ejecutar plan de puzzles
Corre el plan `puzzle_plan_local` usando `PuzzleSpeedAgent` y guarda salidas en `models_local/puzzle/`. Útil para medir tiempos de navegación y resolución de puzzles previos al combate.

In [ ]:
puzzle_runs_local = train_plan(
    agent_key='puzzle',
    plan=puzzle_plan_local,
    default_timesteps=DEFAULT_TIMESTEPS_LOCAL,
    headless=DEFAULT_HEADLESS_LOCAL
)


>>> [PUZZLE] Ejecución 1/1


ModuleNotFoundError: No module named 'torch.utils._sympy'

## 8. Ejecutar plan híbrido
Activa `HybridSageAgent` sobre los escenarios definidos en `hybrid_plan_local`, mezclando comportamientos de combate y navegación y almacenando resultados en `models_local/hybrid/`.

In [ ]:
hybrid_runs_local = train_plan(
    agent_key='hybrid',
    plan=hybrid_plan_local,
    default_timesteps=DEFAULT_TIMESTEPS_LOCAL,
    headless=DEFAULT_HEADLESS_LOCAL
)

## 9. Guardado manual (opcional)
Fragmento de ejemplo para guardar un modelo entrenado con un nombre personalizado. Solo úsalo si traes a la sesión variables como `model`, `AGENT_TYPE`, `SCENARIO_ID` y `PHASE_NAME`; de lo contrario producirá errores.

In [ ]:
# Guardar modelo
save_dir = "models_local"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, f"{AGENT_TYPE}_{SCENARIO_ID}_{PHASE_NAME}")
model.save(save_path)
print(f"Modelo guardado en {save_path}")

NameError: name 'AGENT_TYPE' is not defined

## 10. Comparación con Baseline (PPO v2)

Esta sección permite comparar el desempeño de tus agentes entrenados (Combat, Puzzle, Hybrid) contra un baseline.

**IMPORTANTE - Limitaciones de RAM (16GB):**
- El modelo `poke_26214400.zip` (26M pasos) requiere >10GB solo para cargarlo
- **Alternativa recomendada**: Entrenar tu propio baseline ligero (40k-100k pasos) en lugar de usar el modelo pesado
- O simplemente evaluar solo tus modelos locales sin comparación (ver celda siguiente)

**Alternativa para comparar sin .zip pesado:**
Puedes usar `run_pretrained_interactive.py` como baseline ejecutándolo manualmente y registrando las métricas, pero esta sección automatiza la evaluación de **tus modelos** sin necesidad del baseline gigante.

In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from v2.red_gym_env_v2 import RedGymEnv

def load_baseline_model(path):
    if not os.path.exists(path):
        print(f"No se encontró el modelo baseline en: {path}")
        return None
    try:
        return PPO.load(path)
    except Exception as e:
        print(f"Error cargando baseline: {e}")
        return None

def evaluate_agent_model(model, env, num_episodes=1):
    """Ejecuta episodios de evaluación y retorna métricas promedio."""
    rewards = []
    steps = []
    
    for i in range(num_episodes):
        # Manejar diferentes formatos de reset()
        try:
            reset_result = env.reset()
            obs = reset_result[0] if isinstance(reset_result, tuple) else reset_result
        except Exception as e:
            print(f"Error en reset: {e}")
            continue
        
        done = False
        truncated = False
        total_reward = 0
        step_count = 0
        max_steps = 5000  # Límite de pasos por episodio
        
        while not done and not truncated and step_count < max_steps:
            try:
                # FIX: Asegurar que obs sea dict/array, no tupla
                if isinstance(obs, tuple):
                    obs = obs[0]
                    
                action, _ = model.predict(obs, deterministic=True)
                step_result = env.step(action)
                
                # Manejar diferentes formatos de step()
                if len(step_result) == 5:
                    obs, reward, done, truncated, info = step_result
                elif len(step_result) == 4:
                    obs, reward, done, info = step_result
                    truncated = False
                else:
                    raise ValueError(f"Formato inesperado de step(): {len(step_result)} valores")
                
                # FIX: Manejar VecEnv (arrays) vs Env estándar (escalares)
                if isinstance(done, (list, np.ndarray)):
                    done = done[0] if len(done) > 0 else False
                if isinstance(truncated, (list, np.ndarray)):
                    truncated = truncated[0] if len(truncated) > 0 else False
                if isinstance(reward, (list, np.ndarray)):
                    reward = reward[0] if len(reward) > 0 else 0
                
                # Convertir reward a escalar
                reward_scalar = float(reward.item() if hasattr(reward, 'item') else reward)
                total_reward += reward_scalar
                step_count += 1
                
            except Exception as e:
                print(f"Error en step {step_count}: {e}")
                break
            
        rewards.append(total_reward)
        steps.append(step_count)
        
    return {
        'mean_reward': np.mean(rewards) if rewards else 0,
        'std_reward': np.std(rewards) if rewards else 0,
        'mean_steps': np.mean(steps) if steps else 0
    }

def run_comparison_lightweight(plans_dict, baseline_path=None, headless=True, skip_baseline=False):
    """
    Versión optimizada para RAM limitada (<=16GB con Windows ocupando 10GB).
    skip_baseline=True: Solo evalúa tus modelos locales (recomendado para 16GB RAM)
    """
    results = []
    
    # Cargar Baseline solo si se solicita y existe
    baseline_model = None
    if not skip_baseline and baseline_path:
        print(f"Intentando cargar modelo baseline desde: {baseline_path}")
        baseline_model = load_baseline_model(baseline_path)
        if not baseline_model:
            print("No se pudo cargar el baseline. Solo se evaluarán modelos locales.")
    else:
        print("Modo sin baseline activado (ahorra ~10GB RAM)")
    
    for agent_key, plan in plans_dict.items():
        for entry in plan:
            scenario_id = entry['scenario']
            phase_name = entry.get('phase') or AGENT_REGISTRY[agent_key]['default_phase']
            
            print(f"\n--- Evaluando {agent_key.upper()} en {scenario_id} ({phase_name}) ---")
            
            # 1. Preparar Configuración Común
            try:
                phase = resolve_phase(scenario_id, phase_name)
                state_file_path = ensure_state_file(phase['state_file'])
                env_overrides = build_env_overrides(state_file_path, headless=headless)
                base_config = _base_env_config(env_overrides)
            except Exception as e:
                print(f"❌ Error preparando configuración: {e}")
                continue
            
            # ---------------------------------------------------------
            # 2. Evaluar Agente Local (con su propio wrapper/env)
            # ---------------------------------------------------------
            registry_entry = AGENT_REGISTRY[agent_key]
            agent_config = registry_entry['config_cls'](
                env_config=base_config,
                total_timesteps=1000 
            )
            local_agent_wrapper = registry_entry['agent_cls'](agent_config)
            
            local_model_path = os.path.join(MODELS_DIR, agent_key, f"{scenario_id}_{phase_name}.zip")
            
            # FIX: También buscar versión _stable.zip
            if not os.path.exists(local_model_path):
                stable_path = os.path.join(MODELS_DIR, agent_key, f"{scenario_id}_{phase_name}_stable.zip")
                if os.path.exists(stable_path):
                    local_model_path = stable_path
            
            if os.path.exists(local_model_path):
                print(f"Cargando modelo local: {local_model_path}")
                try:
                    env_local = local_agent_wrapper.make_env()
                    local_agent_wrapper.model = PPO.load(local_model_path)
                    
                    print(f"Ejecutando evaluación...")
                    metrics_local = evaluate_agent_model(local_agent_wrapper.model, env_local)
                    env_local.close()
                    
                    print(f"✅ Local: Reward={metrics_local['mean_reward']:.2f}, Steps={metrics_local['mean_steps']:.0f}")
                    
                    results.append({
                        'Agent': agent_key.upper(),
                        'Scenario': scenario_id,
                        'Phase': phase_name,
                        'Model': 'Local (Specialized)',
                        'Reward': metrics_local['mean_reward'],
                        'Steps': metrics_local['mean_steps']
                    })
                    
                    # Liberar memoria
                    del local_agent_wrapper.model
                    del env_local
                    
                except Exception as e:
                    print(f"❌ Error evaluando local: {e}")
                    import traceback
                    traceback.print_exc()
            else:
                print(f"⚠️ No existe modelo local en {local_model_path}")

            # ---------------------------------------------------------
            # 3. Evaluar Baseline solo si está disponible
            # ---------------------------------------------------------
            if baseline_model:
                print("Evaluando Baseline...")
                try:
                    env_baseline = RedGymEnv(base_config)
                    metrics_baseline = evaluate_agent_model(baseline_model, env_baseline)
                    env_baseline.close()
                    
                    print(f"✅ Baseline: Reward={metrics_baseline['mean_reward']:.2f}, Steps={metrics_baseline['mean_steps']:.0f}")
                    
                    results.append({
                        'Agent': agent_key.upper(),
                        'Scenario': scenario_id,
                        'Phase': phase_name,
                        'Model': 'Baseline (PPO v2)',
                        'Reward': metrics_baseline['mean_reward'],
                        'Steps': metrics_baseline['mean_steps']
                    })
                    
                    del env_baseline
                    
                except Exception as e:
                    print(f"❌ Error evaluando baseline: {e}")

    return pd.DataFrame(results) if results else None


In [ ]:
# ==================== CONFIGURACIÓN DE COMPARACIÓN ====================
# Para sistemas con 16GB RAM (con Windows usando ~10GB):
# skip_baseline=True: Solo evalúa tus modelos (ahorra ~10GB)
# skip_baseline=False: Intenta cargar el baseline (requiere >20GB RAM total)
# ========================================================================

BASELINE_MODEL_PATH = os.path.join(project_path, 'v2', 'runs', 'poke_26214400.zip')

comparison_plans = {
    'combat': combat_plan_local,
    # 'puzzle': puzzle_plan_local,   # Comenta para evaluar menos modelos
    # 'hybrid': hybrid_plan_local,   # Comenta para evaluar menos modelos
}

# IMPORTANTE: skip_baseline=True para ahorrar RAM
df_results = run_comparison_lightweight(
    comparison_plans, 
    baseline_path=BASELINE_MODEL_PATH, 
    headless=True,
    skip_baseline=True  # Cambia a False solo si tienes >24GB RAM
)

if df_results is not None and not df_results.empty:
    print("\n" + "="*60)
    print("           RESULTADOS DE EVALUACIÓN")
    print("="*60)
    print(df_results.to_string(index=False))
    print("="*60)
    
    # Guardar CSV
    csv_path = "evaluacion_modelos_locales.csv"
    df_results.to_csv(csv_path, index=False)
    print(f"\nResultados guardados en: {csv_path}")
else:
    print("\nNo se generaron resultados. Verifica que existan modelos entrenados.")

Modo sin baseline activado (ahorra ~10GB RAM)

--- Evaluando COMBAT en pewter_brock (battle) ---
Cargando modelo local: c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed\models_local\combat\pewter_brock_battle.zip
Error evaluando local: too many values to unpack (expected 2)

No se generaron resultados. Verifica que existan modelos entrenados.


## 11. Entrenar Baseline Ligero (Opcional - Alternativa al modelo pesado)

Si quieres comparar tus agentes especializados con un baseline PPO genérico **sin usar el modelo gigante de 26M pasos**, puedes entrenar tu propio baseline ligero aquí. Este será un modelo estándar de `v2/red_gym_env_v2.py` entrenado con los **mismos 40k pasos** que tus agentes especializados para una comparación justa.

In [ ]:
from stable_baselines3 import PPO
from v2.red_gym_env_v2 import RedGymEnv

def train_lightweight_baseline(scenario_id='pewter_brock', phase_name='battle', timesteps=40_000):
    """
    Entrena un baseline PPO simple (sin wrappers especializados) para comparación justa.
    Usa el mismo número de pasos que tus agentes especializados.
    """
    print(f"\n{'='*60}")
    print(f"   ENTRENANDO BASELINE LIGERO (PPO Genérico)")
    print(f"   Escenario: {scenario_id} | Fase: {phase_name}")
    print(f"   Pasos: {timesteps:,}")
    print(f"{'='*60}\n")
    
    # Preparar configuración del entorno (igual que tus agentes)
    phase = resolve_phase(scenario_id, phase_name)
    state_file_path = ensure_state_file(phase['state_file'])
    env_overrides = build_env_overrides(state_file_path, headless=True)
    base_config = _base_env_config(env_overrides)
    
    # Crear entorno estándar (sin wrappers especializados)
    env = RedGymEnv(base_config)
    
    # Crear modelo PPO con configuración similar a tus agentes
    model = PPO(
        "CnnPolicy",  # Política estándar para imágenes
        env,
        learning_rate=2.5e-4,
        n_steps=1024,
        batch_size=256,
        gamma=0.999,
        verbose=1,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    # Entrenar
    try:
        import tqdm, rich
        model.learn(total_timesteps=timesteps, progress_bar=True)
    except ImportError:
        model.learn(total_timesteps=timesteps)
    
    # Guardar
    baseline_dir = os.path.join(MODELS_DIR, 'baseline_lightweight')
    os.makedirs(baseline_dir, exist_ok=True)
    baseline_path = os.path.join(baseline_dir, f"{scenario_id}_{phase_name}.zip")
    model.save(baseline_path)
    
    print(f"\nBaseline ligero guardado en: {baseline_path}")
    env.close()
    
    return baseline_path

# Entrenar baseline (descomenta para ejecutar)
# baseline_ligero_path = train_lightweight_baseline(
#     scenario_id='pewter_brock',
#     phase_name='battle',
#     timesteps=40_000  # Mismo número de pasos que tus agentes
# )

### Comparar con Baseline Ligero

Una vez entrenado el baseline ligero, puedes compararlo con tus agentes especializados usando esta celda:

In [ ]:
# Ruta al baseline ligero que acabas de entrenar
BASELINE_LIGERO_PATH = os.path.join(project_path, 'models_local', 'baseline_lightweight', 'pewter_brock_battle.zip')

# Comparar (solo si el baseline ligero existe)
if os.path.exists(BASELINE_LIGERO_PATH):
    print("Comparando con Baseline Ligero (entrenado con los mismos 40k pasos)")
    
    df_comparison = run_comparison_lightweight(
        {'combat': combat_plan_local},
        baseline_path=BASELINE_LIGERO_PATH,
        headless=True,
        skip_baseline=False  # Ahora SÍ cargamos el baseline (es ligero)
    )
    
    if df_comparison is not None and not df_comparison.empty:
        print("\n" + "="*70)
        print("     COMPARACIÓN: AGENTE ESPECIALIZADO vs BASELINE LIGERO")
        print("="*70)
        print(df_comparison.to_string(index=False))
        print("="*70)
        
        # Calcular mejora
        if len(df_comparison) == 2:
            reward_especializado = df_comparison[df_comparison['Model'].str.contains('Specialized')]['Reward'].values[0]
            reward_baseline = df_comparison[df_comparison['Model'].str.contains('Baseline')]['Reward'].values[0]
            mejora = ((reward_especializado - reward_baseline) / abs(reward_baseline)) * 100
            print(f"\nMejora del agente especializado: {mejora:+.1f}%")
        
        df_comparison.to_csv("comparacion_especializado_vs_baseline_ligero.csv", index=False)
else:
    print(f"Baseline ligero no encontrado en: {BASELINE_LIGERO_PATH}")
    print("Ejecuta primero la celda anterior para entrenar el baseline ligero.")

In [ ]:
# =================================================================================
# EVALUACIÓN DE AGENTE EN ESCENARIO DE GIMNASIO (HEADLESS + MÉTRICAS)
# VERSIÓN CORREGIDA: Maneja correctamente VecEnv y errores de tipos
# =================================================================================

import json
import time
import sys
import numpy as np
from pathlib import Path
from stable_baselines3 import PPO
from gym_scenarios.gym_metrics import GymMetricsTracker

# Importar direcciones de memoria necesarias para la inyección
PARTY_SIZE_ADDRESS = 0xD163
PARTY_ADDRESSES = [0xD164, 0xD165, 0xD166, 0xD167, 0xD168, 0xD169]
LEVELS_ADDRESSES = [0xD18C, 0xD1B8, 0xD1E4, 0xD210, 0xD23C, 0xD268]
HP_ADDRESSES = [0xD16C, 0xD198, 0xD1C4, 0xD1F0, 0xD21C, 0xD248]
MAX_HP_ADDRESSES = [0xD18D, 0xD1B9, 0xD1E5, 0xD211, 0xD23D, 0xD269]
MONEY_ADDRESS_1 = 0xD347
MONEY_ADDRESS_2 = 0xD348
MONEY_ADDRESS_3 = 0xD349
BADGE_COUNT_ADDRESS = 0xD356
BAG_ITEMS_START = 0xD31E
BAG_ITEM_COUNT = 0xD31D

def get_base_env(env):
    """Obtiene el entorno base (RedGymEnv) de un wrapper o VecEnv."""
    if hasattr(env, 'envs'): # DummyVecEnv/SubprocVecEnv
        env = env.envs[0]
    if hasattr(env, 'unwrapped'):
        return env.unwrapped
    return env

def inject_gym_config(env, config):
    """Inyecta la configuración del equipo e inventario en la memoria del emulador."""
    base_env = get_base_env(env)
    pyboy = base_env.pyboy
    
    def write_mem(addr, val):
        if hasattr(pyboy, "set_memory_value"):
            pyboy.set_memory_value(addr, val & 0xFF)
        else:
            pyboy.memory[addr] = val & 0xFF

    def write_word(addr, val):
        write_mem(addr, (val >> 8) & 0xFF)
        write_mem(addr + 1, val & 0xFF)

    def write_bcd(val):
        return ((val // 10) << 4) | (val % 10)

    print("Inyectando configuración de equipo e inventario...")

    # 1. Equipo
    team = config.get('player_team', [])
    write_mem(PARTY_SIZE_ADDRESS, len(team))
    for i, poke in enumerate(team):
        slot = poke.get('slot', 1) - 1
        if 0 <= slot < 6:
            write_mem(PARTY_ADDRESSES[slot], poke.get('species_id', 0))
            write_mem(LEVELS_ADDRESSES[slot], poke.get('level', 5))
            write_word(HP_ADDRESSES[slot], poke.get('current_hp', 20))
            write_word(MAX_HP_ADDRESSES[slot], poke.get('max_hp', 20))

    # 2. Items
    items = config.get('bag_items', [])
    item_count = min(len(items), 20)
    write_mem(BAG_ITEM_COUNT, item_count)
    for i, item in enumerate(items[:20]):
        base = BAG_ITEMS_START + (i * 2)
        write_mem(base, item.get('item_id', 0))
        write_mem(base + 1, item.get('quantity', 1))
    write_mem(BAG_ITEMS_START + (item_count * 2), 0xFF)

    # 3. Dinero y Medallas
    money = config.get('money', 0)
    write_mem(MONEY_ADDRESS_1, write_bcd(money // 10000))
    write_mem(MONEY_ADDRESS_2, write_bcd((money // 100) % 100))
    write_mem(MONEY_ADDRESS_3, write_bcd(money % 100))
    write_mem(BADGE_COUNT_ADDRESS, config.get('badge_bits', 0))

    # 4. Warp
    start_pos = config.get('start_position', {'x': 4, 'y': 13})
    map_id = config.get('map_id', 0)
    
    print(f"🌀 Programando Warp a Mapa {map_id} ({start_pos['x']}, {start_pos['y']})...")
    write_mem(0xD365, map_id)
    write_mem(0xD366, start_pos['x'])
    write_mem(0xD367, start_pos['y'])
    
    if hasattr(pyboy, "get_memory_value"):
        current_wd72d = pyboy.get_memory_value(0xD12B)
    else:
        current_wd72d = pyboy.memory[0xD12B]
    write_mem(0xD12B, current_wd72d | 0x08)
    write_mem(0xD35D, 0x00)

    return map_id, start_pos

def evaluate_gym_scenario(model_path, scenario_path, headless=True):
    """Ejecuta la evaluación completa de un escenario de gimnasio."""
    
    # Rutas
    state_file = os.path.join(scenario_path, "gym_scenario.state")
    config_file = os.path.join(scenario_path, "team_config.json")
    
    if not os.path.exists(model_path):
        print(f"❌ Modelo no encontrado: {model_path}")
        return None
    if not os.path.exists(state_file):
        print(f"❌ Estado no encontrado: {state_file}")
        return None
    if not os.path.exists(config_file):
        print(f"❌ Configuración no encontrada: {config_file}")
        return None

    # Cargar Configuración
    with open(config_file, 'r') as f:
        team_config = json.load(f)
        
    print(f"\n✅ Evaluando en: {team_config.get('gym_name', 'Unknown Gym')}")
    print(f"   Modelo: {os.path.basename(model_path)}")
    print(f"   Modo Headless: {headless}")

    # Configurar Entorno
    try:
        env_overrides = build_env_overrides(state_file, headless=headless)
        env_overrides['max_steps'] = 2048 * 5 
        base_config = _base_env_config(env_overrides)
        
        # Instanciar Agente
        agent_config = CombatAgentConfig(env_config=base_config, total_timesteps=1000)
        agent_wrapper = CombatApexAgent(agent_config)
        
        # Cargar Modelo
        print("📦 Cargando pesos del modelo...")
        agent_wrapper.model = PPO.load(model_path)
        
        # Crear Entorno
        env = agent_wrapper.make_env()
        
    except Exception as e:
        print(f"❌ Error inicializando entorno/modelo: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    # Reset
    try:
        reset_result = env.reset()
        obs = reset_result[0] if isinstance(reset_result, tuple) else reset_result
    except Exception as e:
        print(f"❌ Error en reset: {e}")
        return None
    
    # Inyectar Configuración
    try:
        target_map, target_pos = inject_gym_config(env, team_config)
        base_env = get_base_env(env)
    except Exception as e:
        print(f"❌ Error inyectando configuración: {e}")
        return None
    
    # Calentamiento para Warp
    print("⏳ Calentando motor para warp (3s)...")
    for _ in range(180):
        base_env.pyboy.tick(1, False)
        if not headless:
            env.render()
            
    # Verificar Warp
    current_map = base_env.read_m(0xD35E)
    if current_map != target_map:
        print(f"⚠️ Warp falló. Mapa: {current_map}, Esperado: {target_map}. Reintentando...")
        inject_gym_config(env, team_config)
        for _ in range(60):
            base_env.pyboy.tick(1, False)
            if not headless: env.render()
        
    # Inicializar Tracker
    tracker = GymMetricsTracker(
        gym_number=team_config.get('gym_number', 1),
        agent_name="CombatApex_Local",
        gym_name=team_config.get('gym_name', "")
    )
    tracker.start()
    
    done = False
    truncated = False
    step_count = 0
    max_steps = env_overrides.get('max_steps', 10000)
    
    print("\n🎮 Iniciando ejecución del agente...")
    
    try:
        while not done and not truncated and step_count < max_steps:
            
            # Asegurar que obs no sea tupla
            if isinstance(obs, tuple):
                obs = obs[0]
                
            # Predecir
            action, _ = agent_wrapper.model.predict(obs, deterministic=True)
            
            # Ejecutar
            step_result = env.step(action)
            
            # Unpacking flexible
            if len(step_result) == 4:
                obs, reward, done, info = step_result
                truncated = False
            else:
                obs, reward, done, truncated, info = step_result
            
            # Manejar VecEnv arrays
            if isinstance(done, (list, np.ndarray)): 
                done = done[0] if len(done) > 0 else False
            if isinstance(truncated, (list, np.ndarray)): 
                truncated = truncated[0] if len(truncated) > 0 else False
            if isinstance(reward, (list, np.ndarray)): 
                reward = reward[0] if len(reward) > 0 else 0
            
            # Renderizar
            if not headless:
                env.render()
                
            # Registrar métricas
            game_state = {
                'x': int(base_env.read_m(0xD362)),
                'y': int(base_env.read_m(0xD361)),
                'map': int(base_env.read_m(0xD35E)),
                'hp': [int(base_env.read_m(HP_ADDRESSES[i])) for i in range(6)],
                'in_battle': bool(base_env.read_m(0xD057) != 0)
            }
            
            action_scalar = int(action.item() if isinstance(action, np.ndarray) else action)
            reward_scalar = float(reward)
            
            tracker.record_step(action_scalar, reward_scalar, game_state)
            
            # Lógica de batalla
            if game_state['in_battle'] and not tracker.battle_started:
                tracker.record_battle_start()
            elif not game_state['in_battle'] and tracker.battle_started:
                tracker.record_battle_end(won=True) 
            
            step_count += 1
            
    except KeyboardInterrupt:
        print("\n⏸️ Evaluación interrumpida por usuario")
    except Exception as e:
        print(f"\n❌ Error durante evaluación: {e}")
        import traceback
        traceback.print_exc()
    finally:
        env.close()
    
    # Finalizar
    tracker.end(success=tracker.battle_won)
    tracker.save_metrics(output_dir="metrics_evaluation")
    
    stats = tracker.get_summary_stats()
    print("\n📊 Resumen de Evaluación:")
    print(json.dumps(stats, indent=2))
    
    # Diagnóstico
    if not stats['battle_won']:
        print("\n⚠️ DIAGNÓSTICO DE FALLO:")
        if stats['unique_tiles_explored'] <= 1:
            print("   🔴 AGENTE INMÓVIL: Solo exploró 1 baldosa.")
        elif stats['battle_steps'] == 0:
            print("   🟠 NO ENTRÓ A BATALLA: El agente se movió pero no inició combate.")
            
    return stats

# --- EJECUTAR EVALUACIÓN ---
MODEL_PATH = os.path.join(project_path, 'models_local', 'combat', 'pewter_brock_battle_stable.zip')
SCENARIO_PATH = os.path.join(project_path, 'gym_scenarios', 'gym1_pewter_brock')

if os.path.exists(MODEL_PATH) and os.path.exists(SCENARIO_PATH):
    stats = evaluate_gym_scenario(MODEL_PATH, SCENARIO_PATH, headless=False)
else:
    print(f"⚠️ Falta modelo o escenario:")
    print(f"   Modelo: {MODEL_PATH} - {'✅' if os.path.exists(MODEL_PATH) else '❌'}")
    print(f"   Escenario: {SCENARIO_PATH} - {'✅' if os.path.exists(SCENARIO_PATH) else '❌'}")



Evaluando en: Pewter City Gym - Brock
Modelo: pewter_brock_battle.zip
Modo Headless: False
Cargando pesos del modelo...
Inyectando configuración de equipo e inventario...
🌀 Programando Warp a Mapa 54 (4, 13)...
Calentando motor para warp (3s)...
Inyectando configuración de equipo e inventario...
🌀 Programando Warp a Mapa 54 (4, 13)...
Calentando motor para warp (3s)...
📊 Métricas iniciadas para CombatApex_Local en Gimnasio 1

Iniciando ejecución del agente...
📊 Métricas iniciadas para CombatApex_Local en Gimnasio 1

Iniciando ejecución del agente...


c:\Users\javi1\anaconda3\envs\pokeenv\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


KeyboardInterrupt: 

In [ ]:
## 📚 Resumen de Mejoras del Notebook

Este notebook ha sido **potenciado** con las siguientes mejoras para evitar errores durante el entrenamiento:

### ✅ Nuevas Características

1. **Diagnóstico Pre-Entrenamiento Automático**
   - Verifica PyTorch, GPU, dependencias, archivos .state y espacio en disco
   - Detecta problemas ANTES de empezar a entrenar

2. **Reparación Automática de PyTorch**
   - Soluciona el error WinError 126 (DLLs corruptas)
   - Reinstala PyTorch con soporte CUDA para RTX 3050

3. **Entrenamiento Simplificado y Robusto**
   - Busca automáticamente el mejor archivo .state
   - Valida cada paso antes de ejecutar
   - Manejo completo de excepciones
   - Configuración estable que garantiza convergencia

4. **Correcciones en Evaluación**
   - Manejo correcto de VecEnv vs Env estándar
   - Fix para deserialización de observaciones
   - Conversión robusta de tipos (numpy → Python)

5. **Mejor Feedback al Usuario**
   - Mensajes claros con emojis y colores
   - Diagnóstico automático de fallos
   - Instrucciones paso a paso para solucionar problemas

### 🎯 Cómo Usar Este Notebook

**Flujo recomendado:**

1. **Ejecuta celdas 1-3**: Configuración inicial y verificación de entorno
2. **Ejecuta celda de Diagnóstico**: Verifica que todo esté listo
3. **Si GPU falla**: Ejecuta celda de Reparación de PyTorch → Reinicia kernel
4. **Ejecuta celda de Entrenamiento Simplificado**: ¡Listo!

**Para entrenamientos largos:**
- Cambia `TIMESTEPS = 500_000` (o más)
- Usa `HEADLESS = True` para evitar ralentizaciones
- Aumenta `NUM_ENVS = 8` si tienes CPU potente

### ⚠️ Problemas Comunes Resueltos

| Problema Original | Solución Implementada |
|-------------------|----------------------|
| WinError 126 (PyTorch corrupto) | Celda de reparación automática |
| value_loss > 1000 | Configuración estable con LR reducido |
| VecEnv unpacking errors | Fix en evaluate_agent_model |
| GPU no detectada | Diagnóstico + instrucciones de reinstalación |
| Archivos .state no encontrados | Búsqueda automática con fallback |
| Kernel crash (OpenMP) | Auto-configuración de KMP_DUPLICATE_LIB_OK |

### 📊 Métricas de Éxito

**Entrenamiento exitoso si ves:**
- ✅ `value_loss` < 100 (idealmente < 10)
- ✅ `explained_variance` > 0.3 (mejorando hacia 0.7+)
- ✅ `approx_kl` < 0.05
- ✅ Reward aumentando gradualmente

**Señales de problema:**
- ❌ `value_loss` > 1000 y creciente
- ❌ `explained_variance` < 0.1
- ❌ Reward constante o decreciente
- ❌ Kernel crash repetido

Si ves señales de problema, ejecuta primero la celda de Diagnóstico.

---

**Última actualización**: Noviembre 2025
**Compatibilidad**: RTX 3050, Windows 11, 16GB RAM, Python 3.10+